In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/surajpal')

Drive already mounted at /surajpal; to attempt to forcibly remount, call drive.mount("/surajpal", force_remount=True).


In [0]:
df = pd.read_csv("/surajpal/My Drive/dataset1.1.csv")
review_text = df["review_text"]
good, bad = df['good'], df['bad']

In [0]:
review_quality = []
if (len(good) == len(bad)):
  for i in range(0, len(good)):
    if (good[i] == 0 and bad[i] == 0):
      review_quality.append([0, 0])
    elif (good[i] == 0 and bad[i] == 1):
      review_quality.append([0, 1])
    elif (good[i] == 1 and bad[i] == 0):
      review_quality.append([1, 0])

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(review_text, review_quality, test_size=0.2, random_state=1)

y_train = np.array(y_train)
y_test = np.array(y_test)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectorizer.fit(x_train)
vectorizer.fit(x_test)

x_train = vectorizer.transform(x_train)
x_test = vectorizer.transform(x_test)

In [0]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(24000, 21508)
(6000, 21508)
(24000, 2)
(6000, 2)


In [0]:
x_train = np.reshape(x_train.toarray(), (24000, 1, 21508))
x_test = np.reshape(x_test.toarray(), (6000, 1, 21508))

In [0]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)
print(x_train)

(24000, 1, 21508)
(6000, 1, 21508)
(24000, 2)
(6000, 2)
[[[0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]]]


In [0]:
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [0]:
classifier = Sequential()
classifier.add(LSTM(units = 6, activation = 'sigmoid', input_shape = (1, len(vectorizer.vocabulary_))))
classifier.add(Dense(units = 2, activation = 'sigmoid'))

In [0]:
classifier.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy')

In [0]:
classifier.fit(x_train, y_train, batch_size = 32, epochs = 1)

Epoch 1/1
24000/24000 [==============================] - 24s 985us/step - loss: 0.4603


In [0]:
predicted_y_test = classifier.predict(x_test)
type(predicted_y_test)

numpy.ndarray

In [0]:
predicted_y_test

array([[0.47926378, 0.4587351 ],
       [0.47913796, 0.4586632 ],
       [0.4751879 , 0.45849434],
       ...,
       [0.48514748, 0.45887816],
       [0.4879972 , 0.45734486],
       [0.4812467 , 0.45882177]], dtype=float32)

In [0]:
for i in range(0, len(predicted_y_test)):
  if (predicted_y_test[i][0] > 0.5):
    predicted_y_test[i][0] = True
  else:
    predicted_y_test[i][0] = False
  if (predicted_y_test[i][1] > 0.5):
    predicted_y_test[i][1] = True
  else:
    predicted_y_test[i][1] = False
print(predicted_y_test)

[[0. 0.]
 [0. 0.]
 [0. 0.]
 ...
 [0. 0.]
 [0. 0.]
 [0. 0.]]


In [0]:
from sklearn import metrics
print("% Accuracy", metrics.accuracy_score(predicted_y_test, y_test))

% Accuracy 0.341
